In [6]:
!pip install osmnx 

  Using cached osmnx-1.7.0-py3-none-any.whl (102 kB)
  Using cached geopandas-0.14.0-py3-none-any.whl (1.1 MB)
  Using cached shapely-2.0.2-cp39-cp39-win_amd64.whl (1.4 MB)
  Using cached fiona-1.9.5-cp39-cp39-win_amd64.whl (22.9 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import osmnx as ox
import networkx as nx
ox.config(log_console=True, use_cache=True)
point = (56.816824376990944, 60.02684867357413)


C:\Users\KinShish\AppData\Local\Temp\ipykernel_13164\3352225024.py:3: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True, use_cache=True)


In [9]:
def find_violations(gdf_bad,gdf_educational):
    violations=[]
    gdf_bad = ox.project_gdf(gdf_bad, to_crs =3857)
    gdf_educational = ox.project_gdf(gdf_educational, to_crs =3857)
    for index, p in gdf_educational.iterrows():
        distance=p["geometry"].distance(gdf_bad["geometry"])
        index_distance = distance.index
        for idx,dist in enumerate(distance):
            if dist <= 100:
                #violations.append([index_distance[idx],dist])
                violations.append({"name":p["name"],"street": p["addr:street"],"bad":get_name(index_distance[idx])})
    return violations          

def find_average_distance(gdf_bad,gdf_park):
    distance_array=[]
    gdf_bad = ox.project_gdf(gdf_bad, to_crs =3857)
    gdf_park = ox.project_gdf(gdf_park, to_crs =3857)
    for p in gdf_park["geometry"]:
        distance=p.distance(gdf_bad["geometry"])
        index_distance = distance.index
        min=10000000
        for idx,dist in enumerate(distance):
            if min>dist:
                min=dist
        distance_array.append([index_distance[idx],min])
        #distance.append([index_distance[idx],get_name(index_distance[idx]),dist])
    average_distance=0
    count=0
    for p in distance_array:
        count+=1
        average_distance+=p[1]  
    return average_distance/count  
    
def find_transport_distance(gdf_transport,gdf_park):
    distance_array=[]
    gdf_transport = ox.project_gdf(gdf_transport, to_crs =3857)
    gdf_park = ox.project_gdf(gdf_park, to_crs =3857)
    for index,p in gdf_park.iterrows():
        distance=p["geometry"].distance(gdf_transport["geometry"])
        index_distance = distance.index
        min=10000000
        for idx,dist in enumerate(distance):
            if min>dist:
                min=dist
        if min>200:
            distance_array.append({"name":p["name"],"street": p["addr:street"]})
                #violations.append([index_distance[idx],get_name(index_distance[idx]),dist])
    return distance_array           

def get_name(name):
    gdf_area_local = ox.geocode_to_gdf(name[0][0]+str(name[1]), by_osmid=True)
    return gdf_area_local["display_name"][0]

In [21]:
import geopandas
tags_all = {
    'amenity': ["bar", "fast_food", "biergarten", "pub", "marketplace","school","kindergarten","university","college","music_school",
                "language_school","dancing_school","trade_school"],
    "building": "riding_hall",
    "highway":"cycleway",
    "leisure":["swimming_pool","stadium","fitness_centre","sports_hall","sports_centre","pitch","park","playground","picnic_table",
               "nature_reserve","track","fitness_centre","fitness_station","stadium","outdoor_seating","golf_course","garden","common",
               "sports_hall","dog_park","resort","horse_riding","fishing","water_park","beach_resort","miniature_golf","ice_rink",
               "bird_hide","swimming_area","bandstand","schoolyard","disc_golf_course","hackerspace","summer_camp","indoor_play","trampoline_park",
               "bathing_place","wildlife_hide","barefoot","paddling_pool","village_swing","sunbathing","foot_bath","soccer_golf","wellness"],
    "shop":["greengrocer","agrarian","farm", "cigarettes", "e-cigarette", "vape", "vape_shop", "tobacco", "wine","brewing_supplies",
                 "beverages", "alcohol", "beer", "beverages", "pizza", "fast_food","kiosk","convenience"],
    "cuisine": ["pizza", "burger", "shawarma"]
    
    }
tags_good = {
    'amenity': "marketplace",
    "building": "riding_hall",
    "highway":"cycleway",
    "leisure":["swimming_pool","stadium","fitness_centre","sports_hall","sports_centre","pitch","park","playground","picnic_table",
               "nature_reserve","track","fitness_centre","fitness_station","stadium","outdoor_seating","golf_course","garden","common",
               "sports_hall","dog_park","resort","horse_riding","fishing","water_park","beach_resort","miniature_golf","ice_rink",
               "bird_hide","swimming_area","bandstand","schoolyard","disc_golf_course","hackerspace","summer_camp","indoor_play","trampoline_park",
               "bathing_place","wildlife_hide","barefoot","paddling_pool","village_swing","sunbathing","foot_bath","soccer_golf","wellness"],
    "shop":["greengrocer","agrarian","farm"]}
tags_bad = {
        'amenity': ["bar", "fast_food", "biergarten", "pub"],
        "shop": ["cigarettes", "e-cigarette", "vape", "vape_shop", "tobacco", "wine","brewing_supplies",
                 "beverages", "alcohol", "beer", "beverages", "pizza", "fast_food","kiosk","convenience"],
        "cuisine": ["pizza", "burger", "shawarma"]}
tags_park = {"leisure":["park","wildlife_hide","dog_park"]}
tags_cigarettes = {"shop":["cigarettes","e-cigarette","vape","vape_shop","tobacco","kiosk","convenience"]}
tags_alcohol = {'amenity': ["bar","biergarten","pub"], "shop":["wine","beverages","alcohol","beer","brewing_supplies","kiosk","convenience"]}
tags_educational = {'amenity': ["school","kindergarten","university","college","music_school","language_school","dancing_school","trade_school"]}
tags_transport = {'highway': "bus_stop", "public_transport": ["platform", "stop_position"]}
result=dict();
#display(area.head())

try:
    gdf_good = ox.features_from_point(point, tags_good)
except:
    gdf_good=[]
try:
    gdf_bad = ox.features_from_point(point, tags_bad)
except:
    gdf_bad=[]
try:
    gdf_educational = ox.features_from_point(point, tags_educational)
    violations=find_violations(gdf_bad,gdf_educational.copy(deep=True))
except:
    gdf_educational=[]
    violations=[]


result["good"]=len(gdf_good)
result["positive"]=len(gdf_good)*0.5-len(gdf_bad)
result["educational"]=len(gdf_educational)
result["bad"]=len(gdf_bad)
result["violations"]={"len":len(violations)}#,"data":violations}
result["rating"]=0
if result["positive"]>0:
    result["rating"]+=60
if result["violations"]["len"]==0:
    result["rating"]+=40  

display(result)    

{'good': 1,
 'positive': 0.5,
 'educational': 0,
 'bad': 0,
 'violations': {'len': 0},
 'rating': 100}

In [36]:
gdf_all = ox.features_from_point(point, tags_all, dist=5000)
display(gdf_all.head())
gdf_good=gdf_all[gdf_all.amenity=="marketplace"]
display(gdf_good.head())


name         shop  \
element_type osmid                                         
node         1283830535            Провизия  convenience   
             2971757817         Зелёный мыс          NaN   
             4283645495            продукты  convenience   
             4283661889              молоко  convenience   
             4283669808  Заводская столовая          NaN   

                                          geometry leisure    amenity  \
element_type osmid                                                      
node         1283830535  POINT (59.97554 56.79449)     NaN        NaN   
             2971757817  POINT (60.09172 56.81788)  resort        NaN   
             4283645495  POINT (59.97238 56.79439)     NaN        NaN   
             4283661889  POINT (59.97499 56.79456)     NaN        NaN   
             4283669808  POINT (59.97160 56.79500)     NaN  fast_food   

                                           cuisine drive_through  fast_food  \
element_type osmid                                                            
node         1283830535                        NaN           NaN        NaN   
             2971757817                        NaN           NaN        NaN   
             4283645495                        NaN           NaN        NaN   
             4283661889                        NaN           NaN        NaN   
             4283669808  усовершенствованный_совок            no  cafeteria   

                        addr:city addr:housenumber  ... opening_hours nodes  \
element_type osmid                                  ...                       
node         1283830535       NaN              NaN  ...           NaN   NaN   
             2971757817       NaN              NaN  ...           NaN   NaN   
             4283645495       NaN              NaN  ...           NaN   NaN   
             4283661889       NaN              NaN  ...           NaN   NaN   
             4283669808       NaN              NaN  ...           NaN   NaN   

                        area piste:type phone source building landuse ways  \
element_type osmid                                                           
node         1283830535  NaN        NaN   NaN    NaN      NaN     NaN  NaN   
             2971757817  NaN        NaN   NaN    NaN      NaN     NaN  NaN   
             4283645495  NaN        NaN   NaN    NaN      NaN     NaN  NaN   
             4283661889  NaN        NaN   NaN    NaN      NaN     NaN  NaN   
             4283669808  NaN        NaN   NaN    NaN      NaN     NaN  NaN   

                        type  
element_type osmid            
node         1283830535  NaN  
             2971757817  NaN  
             4283645495  NaN  
             4283661889  NaN  
             4283669808  NaN  

[5 rows x 25 columns]

,,name,shop,geometry,leisure,amenity,cuisine,drive_through,fast_food,addr:city,addr:housenumber,...,opening_hours,nodes,area,piste:type,phone,source,building,landuse,ways,type
element_type,osmid,,,,,,,,,,,,,,,,,,,,,
